In [1]:
print("ok")

ok


In [2]:
import os
os.chdir(r"r:\Sem-5\MiniProject\MediBot2.0\research")
print("Now in:", os.getcwd())


Now in: r:\Sem-5\MiniProject\MediBot2.0\research


In [58]:

%pwd

'r:\\Sem-5\\MiniProject\\MediBot2.0\\research'

In [ ]:
%pip install langchain pymupdf langchain-community


Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [1]:

from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


c:\Users\MMS\miniconda3\envs\MediBot2.0\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.document_loaders import PyPDFLoader
# Extract Data from the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader, DirectoryLoader

extracted_data = load_pdf_file(data='Data/')

In [4]:
#Split data into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)
print("Length of the Text Chunks", len(text_chunks))

Length of the Text Chunks 5859


In [6]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


In [7]:
# Install embeddings from hugging face
def download_hugging_face_embeddings():
    embeddings= HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install --upgrade sentence-transformers huggingface-hub
embeddings = download_hugging_face_embeddings()

  Using cached huggingface_hub-1.2.1-py3-none-any.whl.metadata (13 kB)
Note: you may need to restart the kernel to use updated packages.


C:\Users\MMS\AppData\Local\Temp\ipykernel_24708\1009292861.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [17]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [18]:
# from dotenv import load_dotenv
# load_dotenv()

In [19]:
# import os
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
# OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")

In [20]:
# from pinecone import Pinecone
# import os

# # Use the notebook variable if defined, otherwise fallback to environment variable
# api_key = globals().get("PINECONE_API_KEY") or os.environ.get("PINECONE_API_KEY")
# if not api_key:
# 	raise EnvironmentError("PINECONE_API_KEY not found. Define PINECONE_API_KEY in the notebook or set it in the environment (.env).")

# pc = Pinecone(api_key=api_key)
# index = pc.Index("medibot2")

In [21]:
%pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [22]:
# from pinecone import Pinecone

# pc = Pinecone(api_key="pcsk_t28Du_GP842FwprduRzSFoGzJdGMNC92pSQatia7T3qa1Tjf1cnnWwKnzSgkcBpphYAJ9")
# index = pc.Index("medibot")

In [23]:
# #Embed each chunk and insert embedding into pinecone index.
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name="medibot",
#     embedding=embeddings,
# )

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")

In [13]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc= Pinecone(api_key= PINECONE_API_KEY)

index_name= "medibot2"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medibot2",
    "metric": "cosine",
    "host": "medibot2-i7ceoys.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [14]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY

In [28]:
%pip install --upgrade langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [15]:

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [16]:
# Load Existing Index using variables already defined in the notebook:
# - index_name (should be "medibot2")
# - embeddings (HuggingFaceBgeEmbeddings instance)
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [17]:
docsearch

In [18]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [19]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='65d869c6-dcdc-4a37-ad5c-0c4225298437', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='01bbb4e3-f2df-4b9f-8b8c-8e8657371589', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nat

In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="meta-llama/llama-3.1-70b-instruct",   # Better than GPT-OSS
    temperature=0.4,
    max_tokens=500,
    openai_api_key=os.environ["OPENROUTER_API_KEY"],
    openai_api_base="https://openrouter.ai/api/v1"
)


In [40]:
# In a Jupyter/Colab notebook use %pip so the kernel sees the new packages
%pip install --upgrade langchain-classic langchain-core
# or, if you prefer modern v1 only (no compatibility layer)
# %pip install --upgrade langchain langchain-core openai


Note: you may need to restart the kernel to use updated packages.


In [21]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI   # or whichever chat model package you use


system_prompt = (
    "You are MediBot, assistant chatbot for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know thw answer, say that you "
    "don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)

In [22]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [23]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a skin disorder in which the sebaceous glands become inflamed. It is a general term that refers to a condition characterized by inflammation of the sebaceous glands.


In [45]:
response = rag_chain.invoke({"input": "I have fever what are the medicines i can take for nomal fever?"})
print(response["answer"])

According to the provided context, you can take Acetaminophen (Tylenol) to relieve pain, aches, and fever over 101°F (38.3°C).


In [53]:
response = rag_chain.invoke({"input": "who are you?"})
print(response["answer"])

I am MediBot, a chatbot designed to assist with question-answering tasks, specifically in the context of medical and health-related topics.


In [54]:
response = rag_chain.invoke({"input": "Mujhe cold and cough hai mai kya karun?"})
print(response["answer"])

Ayurveda ke anusaar, cold aur cough ko kapha dosha ke asantulan ke roop mein dekha jaata hai. Aap garam paani pe sakte hain, tulsi aur adrak ki chai pi sakte hain, aur swaadisht aur garmaahut bhojan kha sakte hain.


In [56]:
response = rag_chain.invoke({"input": "i have a symptoms like red patches on the skin, has eruption and water discharge from it. What is this disease called?"})
print(response["answer"])

Based on the symptoms you provided, it seems like you might have Eczema. Eczema is characterized by inflammation of the skin with itching and a rash, which can have blisters that ooze and form crusts.
